![Notebook Banner](llama3_banner.png)

## Set Up Docker Login (Executing on A Terminal Locally)

Replace <your api key> with your own NGC API Key

```bash
export NGC_CLI_API_KEY="nvapi-xxx"
echo "$NGC_CLI_API_KEY" | docker login nvcr.io --username '$oauthtoken' --password-stdin
```

## Deploy NIM (Executing on this Jupyter Notebook)

In [ ]:
%%bash

export NGC_API_KEY="nvapi-xxx"

export CONTAINER_NAME="Llama3-8B-Instruct"
export IMG_NAME="nvcr.io/nim/meta/llama3-8b-instruct:1.0.3"
export LOCAL_NIM_CACHE=~/.cache/nim
mkdir -p $LOCAL_NIM_CACHE

docker run -d --rm --name=$CONTAINER_NAME \
    --runtime=nvidia \
    --gpus all \
    --shm-size=16GB \
    -e NGC_API_KEY \
    -v $LOCAL_NIM_CACHE:/opt/nim/.cache \
    -u $(id -u) \
    -p 8000:8000 \
    $IMG_NAME

036841bc33b195208da4220918b06ed3bf31e4d0407db53ac346a5d230a8c18c


### Make Sure the Contianer is Up and Running
You should see this line in the log.

INFO 07-30 10:39:50.289 server.py:214] Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)

In [7]:
!docker logs Llama3-8B-Instruct


== NVIDIA Inference Microservice LLM NIM ==

NVIDIA Inference Microservice LLM NIM Version 1.0.3
Model: meta/llama3-8b-instruct

Container image Copyright (c) 2016-2024, NVIDIA CORPORATION & AFFILIATES. All rights reserved.

This NIM container is governed by the NVIDIA AI Product Agreement here:
https://www.nvidia.com/en-us/data-center/products/nvidia-ai-enterprise/eula/.
A copy of this license can be found under /opt/nim/LICENSE.

The use of this model is governed by the AI Foundation Models Community License
here: https://docs.nvidia.com/ai-foundation-models-community-license.pdf.

ADDITIONAL INFORMATION: Meta Llama 3 Community License, Built with Meta Llama 3.
A copy of the Llama 3 license can be found under /opt/nim/MODEL_LICENSE.

2025-07-30 10:39:35,710 [INFO] PyTorch version 2.2.2 available.
2025-07-30 10:39:36,342 [WARNING] [TRT-LLM] [W] Logger level already set from environment. Discard new verbosity: error
2025-07-30 10:39:36,342 [INFO] [TRT-LLM] [I] Starting TensorRT-LLM in

## Start Inferencing


In [12]:
pip install openai

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [1]:
from openai import OpenAI
client = OpenAI(
    # base_url='https://integrate.api.nvidia.com/v1',
    base_url='http://0.0.0.0:8000/v1',
    api_key="$NGC_API_KEY"
)
completion = client.chat.completions.create(
    model='meta/llama3-8b-instruct',
    messages=[{"role":"user", "content":"Write a limerick about the wonders of GPU computing."}],
    temperature=0.5,
    top_p=1,
    max_tokens=1024,
    stream=True
)
for chunk in completion:
    if chunk.choices[0].delta.content is not None:
        print(chunk.choices[0].delta.content, end="")

There once was a GPU so fine,
Whose computing powers were truly divine.
It processed with ease,
Complex tasks with expertise,
And solved problems in a short time!

## Stop the Container

In [1]:
!docker ps -a && \
docker stop Llama3-8B-Instruct

CONTAINER ID   IMAGE                                       COMMAND                  CREATED         STATUS         PORTS                                       NAMES
036841bc33b1   nvcr.io/nim/meta/llama3-8b-instruct:1.0.3   "/opt/nvidia/nvidia_…"   4 minutes ago   Up 4 minutes   0.0.0.0:8000->8000/tcp, :::8000->8000/tcp   Llama3-8B-Instruct
Llama3-8B-Instruct
